
For our purpose we have used the Data and models stored in Drive. But we have also provided the models in Github which can be imported and used here and provided the drive link too.

In [34]:
import wave
import gc
import random
import librosa
import librosa.display
import IPython
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.client import device_lib
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import multiprocessing as mp
import os
import cv2

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
def extract_sample(n_way, n_support,n_unlabel, n_query, datax, datay,dataunlabelx,dataunlabely):
  sample = []
  sample_unlabel=[]
  K = np.random.choice(np.unique(datay),n_way, replace = False)
  # K = np.random.choice(np.unique(datay),n_way)
  #print(np.unique(datay))
  #print(K)
  for cls in K:
    datax_cls = datax[datay == cls]
    datax_cls_unlabel=dataunlabelx[dataunlabely == cls]
    perm = np.random.permutation(datax_cls)
    permunlabel = np.random.permutation(datax_cls_unlabel)
    sample_cls = perm[:(n_support+n_query)]
    sample_cls_unlabel=permunlabel[:n_unlabel]
    sample_unlabel.append(sample_cls_unlabel)
    sample.append(sample_cls)
  sample = np.array(sample)
  #print(sample)
  sample_unlabel=np.array(sample_unlabel)
  sample = torch.from_numpy(sample).float()
  sample_unlabel=torch.from_numpy(sample_unlabel).float()
  sample = sample.permute(0,1,4,2,3)
  sample_unlabel = sample_unlabel.permute(0,1,4,2,3)
  return({
      'images': sample,
      'n_way': n_way,
      'n_support': n_support,
      'n_unlabel':n_unlabel,
      'n_query': n_query,
      'unlabelimages':sample_unlabel
      })

In [ ]:
class Flatten(nn.Module):
  def __init__(self):
    super(Flatten, self).__init__()

  def forward(self, x):
    return x.view(x.size(0), -1)

def load_protonet_conv(**kwargs):
  x_dim = kwargs['x_dim']
  hid_dim = kwargs['hid_dim']
  z_dim = kwargs['z_dim']
  def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2))
  encoder = nn.Sequential(
    conv_block(x_dim[0], hid_dim),
    conv_block(hid_dim, hid_dim),
    conv_block(hid_dim, hid_dim),
    conv_block(hid_dim, z_dim),
    Flatten())
  return ProtoNet(encoder)

In [ ]:
class ProtoNet(nn.Module):
  def __init__(self, encoder):
    super(ProtoNet, self).__init__()
    self.encoder = encoder.cuda()

  def set_forward_loss(self, sample):
    sample_images = sample['images'].cuda()
    sample_unlabel=sample['unlabelimages'].cuda()
    n_way = sample['n_way']
    n_support = sample['n_support']
    n_query = sample['n_query']
    n_unlabel=sample['n_query']
    x_support = sample_images[:, :n_support]
    x_query = sample_images[:, n_support:]
    x_unlabel=sample_unlabel
    #target indices are 0 ... n_way-1
    target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
    target_inds = Variable(target_inds, requires_grad=False)
    target_inds = target_inds.cuda()
    #print(target_inds)
    ########################FOR UNLABELLED SET 
    xun = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_unlabel.contiguous().view(n_way * n_unlabel, *x_unlabel.size()[2:])], 0)
    zun = self.encoder.forward(xun)
    zun_dim = zun.size(-1)
    zun_protostar = zun[:n_way*n_support].view(n_way, n_support, zun_dim).mean(1)
    zun_protostarcopy=zun[:n_way*n_support].view(n_way, n_support, zun_dim).mean(1)
    #zun_unlabelquery = zun[n_way*n_unlabel:]  
    zun_unlabelquery = zun[n_way*n_support:]
    name=zun_unlabelquery.view(n_way, n_support, zun_dim)
    distsstar = euclidean_dist(zun_unlabelquery, zun_protostar) 
    # print( distsstar.shape)/
    #log_p_ystar = F.log_softmax(-distsstar,dim=1).view(n_way, n_support, -1)
    log_p_ystar = F.log_softmax(-distsstar,dim=1).view(n_way, n_unlabel, -1)
    loss_valstar = -log_p_ystar.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_ystar.max(2)
    #print(y_hat)
    acc_valold1 = torch.eq(y_hat, target_inds.squeeze()).float().mean()
    m = nn.Softmax(dim=1)
    output = m(-distsstar).view(n_way, n_unlabel, -1)
    # print(output.shape)
    prototype=losss(y_hat,x_unlabel,name,zun_dim,n_way,zun_protostarcopy,n_support,output)
    ##################FOR UNLABELLEDSET+SUPPORT SET PROTOTYPE FOUND---> now using new prototype for the calculation
    x = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])], 0)
    z=self.encoder.forward(x)
    z_query = z[n_way*n_support:] #for query set 
    ####FOR QUERY SET WITHOUT UNLABELED
    distold=euclidean_dist(z_query,zun_protostar)# old prototype
    #log_p_ystarold = F.log_softmax(-distold,dim=1).view(n_way, n_support, -1) 
    log_p_ystarold = F.log_softmax(-distold,dim=1).view(n_way, n_query, -1)
    loss_valold = -log_p_ystarold.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hatold = log_p_ystarold.max(2)
    acc_valold = torch.eq(y_hatold, target_inds.squeeze()).float().mean()
    # print("accuracy on query set without adding unlabeled")
    # print(acc_valold)
    # print(prototype)
    #### END OF QUERY SET WITHOUT UNLABELED
    #### FOR QUERY SET WITH UNLABELED
    dists = euclidean_dist(z_query, prototype) # distance between query and new prototype
    log_p_ystar = F.log_softmax(-dists,dim=1).view(n_way, n_query, -1)
    loss_val = -log_p_ystar.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hatfinal = log_p_ystar.max(2)
    acc_val = torch.eq(y_hatfinal, target_inds.squeeze()).float().mean()
    #print(target_inds.squeeze())
    # print(acc_val)
     #### FOR END OF QUERY SET WITH UNLABELED
    return loss_val, {
        'loss': loss_val.item(),
        'acc': acc_val.item(),
        'y_hat': y_hatfinal
        }

In [ ]:

#making of new prototype
def losss(y_hat,x_unlabel,name,zdim,n_way,zun_protostarcopy,n_support,output):
  ar=torch.zeros(n_way,zdim).cuda()
  j1=0
  sizearray=torch.zeros(n_way)
  p=0
  for j1 in range(n_way):
    zun_protostarcopy[p]=zun_protostarcopy[p]*n_support
    #print(len(zun_protostarcopy[p]))
    sizearray[p]=sizearray[p]+n_support
    p=p+1
  j=0 
  #print(y_hat)
  for k in y_hat:
    i=0
    for kk in k:
      for y in range(len(zun_protostarcopy)):
        zun_protostarcopy[y]=zun_protostarcopy[y]+name[j][i]*output[j][i][y]
        sizearray[y]=sizearray[y]+output[j][i][y]
      i=i+1
    j=j+1
  k=0
  for l in sizearray:
    zun_protostarcopy[k]=zun_protostarcopy[k]/int(l)
    k=k+1
  return zun_protostarcopy

In [ ]:
def euclidean_dist(x, y):
  n = x.size(0)
  m = y.size(0)
  d = x.size(1)
  assert d == y.size(1)

  x = x.unsqueeze(1).expand(n, m, d)
  y = y.unsqueeze(0).expand(n, m, d)

  return torch.pow(x - y, 2).sum(2)

In [ ]:
from tqdm import tqdm_notebook
from tqdm import tnrange

In [ ]:
def train(model, optimizer, train_x, train_y, n_way, n_support,n_unlabel, n_query, max_epoch, epoch_size,Xunlabel,Yunlabel):
  #divide the learning rate by 2 at each epoch, as suggested in paper
  scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
  epoch = 0 #epochs done so far
  stop = False #status to know when to stop

  while epoch < max_epoch and not stop:
    running_loss = 0.0
    running_acc = 0.0
    for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch+1)):
      sample = extract_sample(n_way, n_support,n_unlabel, n_query, train_x, train_y,Xunlabel,Yunlabel)
      optimizer.zero_grad()
      loss, output = model.set_forward_loss(sample)
      running_loss += output['loss']
      running_acc += output['acc']
      loss.backward()
      optimizer.step()
    epoch_loss = running_loss / epoch_size
    epoch_acc = running_acc / epoch_size
    print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch+1,epoch_loss, epoch_acc))
    epoch += 1
    scheduler.step()
    

SECTION FOR TRAINING TESTING ON DATASET D1 URBANSOUND8K AND FSD DATASET



In [ ]:
Xtrain=np.load("/content/drive/MyDrive/urbansound8k/300data/final/xtrainlabel.npy")
Ytrain=np.load("/content/drive/MyDrive/urbansound8k/300data/final/ytrainlabel.npy")
Xunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/final/xtrainunlabel.npy")
Yunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/final/ytrainunlabel.npy")

In [ ]:
Xunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/xtestunlabel.npy')
Yunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/ytestunlabel.npy')
Xtest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/xtestlabel.npy')
Ytest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/ytestlabel.npy')

In [ ]:
%%time
import torch
from tensorflow import keras
model = load_protonet_conv(
    x_dim=(4,130,130),
    hid_dim=64,
    z_dim=64,
    )

optimizer = optim.Adam(model.parameters(), lr = 0.003)
n_way = 5
n_support = 5
n_query = 5
n_unlabel= 5
train_x = Xtrain
train_y = Ytrain
max_epoch = 10
epoch_size = 2000
train(model, optimizer, train_x, train_y, n_way, n_support, n_unlabel, n_query, max_epoch, epoch_size,Xunlabel,Yunlabel)
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



Epoch 1 -- Loss: 3.9485 Acc: 0.5697



Epoch 2 -- Loss: 0.7297 Acc: 0.7240



Epoch 3 -- Loss: 0.5325 Acc: 0.7954



Epoch 4 -- Loss: 0.4411 Acc: 0.8280


...


Epoch 6 -- Loss: 0.3340 Acc: 0.8711



Epoch 7 -- Loss: 0.2965 Acc: 0.8859



Epoch 8 -- Loss: 0.2783 Acc: 0.8943



Epoch 9 -- Loss: 0.2655 Acc: 0.8985



Epoch 10 -- Loss: 0.2704 Acc: 0.8968
CPU times: user 48min 58s, sys: 5min 44s, total: 54min 42s
Wall time: 55min 43s


In [ ]:
torch.save(model,"/content/drive/MyDrive/urbansound8k/300data/final/3.1COMBINE.pt") 

In [ ]:
model=torch.load("/content/drive/MyDrive/urbansound8k/300data/final/3.1COMBINE.pt")

In [ ]:
def test(model, test_x, test_y, n_way, n_support,n_unlabel, n_query, test_episode,Xunlabel,Yunlabel):
  running_loss = 0.0
  running_acc = 0.0
  for episode in tnrange(test_episode):
    sample = extract_sample(n_way, n_support,n_unlabel, n_query, test_x, test_y,Xunlabel,Yunlabel)
    loss, output = model.set_forward_loss(sample)
    running_loss += output['loss']
    running_acc += output['acc']
  avg_loss = running_loss / test_episode
  avg_acc = running_acc / test_episode
  print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

In [ ]:
##TESTING ON DATASET D1 WITH FSD AND URBANSOUND8K
n_way = 5
n_support = 5
n_query = 5
n_unlabel= 5
test_x = Xtest
test_y = Ytest
test_episode = 2000
test(model, test_x, test_y, n_way, n_support,n_unlabel, n_query, test_episode,Xunlabeltest,Yunlabeltest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.



Test results -- Loss: 0.8150 Acc: 0.6782


SECTION FOR TRAINING AND TESTING ON THE D2 DATASET THAT IS ONLY FSD DATASET


In [ ]:
Xtrain=np.load("/content/drive/MyDrive/urbansound8k/300data/xtrainlabelFSD.npy")
Ytrain=np.load("/content/drive/MyDrive/urbansound8k/300data/ytrainlabelFSD.npy")
Xunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/xtrainunlabelFSD.npy")
Yunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/ytrainunlabelFSD.npy")

In [ ]:
print(Xtrain.shape)
print(Ytrain.shape)
print(Xunlabel.shape)
print(Yunlabel.shape)

(1680, 130, 130, 4)
(1680,)
(2520, 130, 130, 4)
(2520,)


In [ ]:
Xunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/xtestunlabelFSD.npy')
Yunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/ytestunlabelFSD.npy')
Xtest = np.load('/content/drive/MyDrive/urbansound8k/300data/xtestlabelFSD.npy')
Ytest = np.load('/content/drive/MyDrive/urbansound8k/300data/ytestlabelFSD.npy')

In [ ]:
print(Xtest.shape)
print(Ytest.shape)
print(Xunlabeltest.shape)
print(Yunlabeltest.shape)

(720, 130, 130, 4)
(720,)
(1080, 130, 130, 4)
(1080,)


In [ ]:
%%time
##TRAIN ON D2 FSD ONLY
import torch
from tensorflow import keras
model = load_protonet_conv(
    x_dim=(4,130,130),
    hid_dim=64,
    z_dim=64,
    )

optimizer = optim.Adam(model.parameters(), lr = 0.003)
n_way = 5
n_support = 5
n_query = 5
n_unlabel= 5
train_x = Xtrain
train_y = Ytrain
max_epoch = 10
epoch_size = 2000
train(model, optimizer, train_x, train_y, n_way, n_support, n_unlabel, n_query, max_epoch, epoch_size,Xunlabel,Yunlabel)
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



Epoch 1 -- Loss: 6.3178 Acc: 0.4230



Epoch 2 -- Loss: 1.0306 Acc: 0.5872



Epoch 3 -- Loss: 0.8518 Acc: 0.6665



Epoch 4 -- Loss: 0.7396 Acc: 0.7155


...


Epoch 6 -- Loss: 0.5550 Acc: 0.7914



Epoch 7 -- Loss: 0.4950 Acc: 0.8095



Epoch 8 -- Loss: 0.4599 Acc: 0.8261



Epoch 9 -- Loss: 0.4470 Acc: 0.8319



Epoch 10 -- Loss: 0.4362 Acc: 0.8346
CPU times: user 48min 13s, sys: 5min 36s, total: 53min 50s
Wall time: 54min 50s


In [ ]:
torch.save(model,"/content/drive/MyDrive/urbansound8k/300data/final/3.1FSD.pt")## SAVING FSD ON D2 MODEL

In [ ]:
model=torch.load("/content/drive/MyDrive/urbansound8k/300data/final/3.1FSD.pt")

In [ ]:
def test(model, test_x, test_y, n_way, n_support,n_unlabel, n_query, test_episode,Xunlabel,Yunlabel):
  running_loss = 0.0
  running_acc = 0.0
  for episode in tnrange(test_episode):
    sample = extract_sample(n_way, n_support,n_unlabel, n_query, test_x, test_y,Xunlabel,Yunlabel)
    loss, output = model.set_forward_loss(sample)
    running_loss += output['loss']
    running_acc += output['acc']
  avg_loss = running_loss / test_episode
  avg_acc = running_acc / test_episode
  print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

In [ ]:
##TESTING ON DATASET D2 WITH FSD 
n_way = 5
n_support = 5
n_query = 5
n_unlabel= 5
test_x = Xtest
test_y = Ytest
test_episode = 5000
test(model, test_x, test_y, n_way, n_support,n_unlabel, n_query, test_episode,Xunlabeltest,Yunlabeltest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



Test results -- Loss: 1.4509 Acc: 0.4623


ENDS
